# Modelling with lasagne

## Import and options

In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
import pickle

import h5py
import dask
# from dask import array as da
from dask import dataframe as dd
# from dask import delayed
# from dask.multiprocessing import get
import pandas as pd
import pathlib2 as pl
import mmh3  # The hash function used to hash sites. See the preprocessor script.

In [3]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 1000)

# dask.set_options(get=get);  # Due to a bug we can't read files in different processes so set this option after reading.

In [4]:
RANDOM_SEED = 26715
CHUNK_SIZE = int(2e5)

MAX_BATCHES = 1000
MAX_BATCH_LENGTH = 100  # Maximum number os day-long measurement sequence (of one site) per batch
MAX_SEQ_LENGTH = 24*60 # Maximum number of measurements per site per day
SUBSEQ_LENGTH = 60  # 1 measurement/min a subsquence of 1h

## Loading data into dask dataframes

Our preprocessor supports output into `numpy` `arrays` and `pandas` `DataFrames` and `scikit-learn` supports the latter.

In [5]:
DF_DIR = pl.Path('/Volumes/CompanionEx/Data/dfs_pandas/PP_TS_2016-05-24-00_2016-06-01-00*.hdf')
str(DF_DIR)

'/Volumes/CompanionEx/Data/dfs_pandas/PP_TS_2016-05-24-00_2016-06-01-00*.hdf'

In [6]:
data = dd.read_hdf(str(DF_DIR), key='dataset', chunksize=CHUNK_SIZE)
# data = pd.read_hdf('/Volumes/CompanionEx/Data/dfs_pandas/PP_TS_2016-05-24-00_2016-06-01-00_0-200_20160624101922.hdf')
# data.head()

day_counts = dd.read_hdf(str(DF_DIR), key='day_counts', chunksize=CHUNK_SIZE)
site_counts = dd.read_hdf(str(DF_DIR), key='site_counts', chunksize=CHUNK_SIZE)

In [7]:
with open('../selected_sites.pkl', mode='rb') as fname:
    sites = pickle.load(fname)

print(len(sites))
sites[:5]

17650


['rws01_monibas_0010vwa0065ra',
 'rws01_monibas_0010vwa0223ra',
 'rws01_monibas_0010vwa0248ra',
 'rws01_monibas_0010vwa0269ra',
 'rws01_monibas_0010vwa0286ra']

In [8]:
from random import sample, seed, shuffle
seed(RANDOM_SEED)

In [9]:
samples_sites = sample(sites, 3000)

In [10]:
# print(len(data))
data = data.query('site in {}'.format(samples_sites))
# print(len(data))

In [15]:
# If you're using commits of the preprocessor after 10 Aug this is already done for you and you can skip it. 
# def create_day(data):
#     datetime_index = data.index.get_level_values('datetime_start')
#     datetime_index = pd.DatetimeIndex(datetime_index)

#     data['day'] = datetime_index.to_period(freq='d')

We can apply a query at this stage to limit the dataset.

## Split into train, test and validation sets

We will split by selecting a day as test and another as validation sets.

In [ ]:
test_day = day_counts.
.keys()[-2]
validation_day = days.keys()[-3]
print(test_day, validation_day)

In [ ]:
from datetime import date, datetime, timedelta

def filter_for_day(data, day, complement=False):
    datetime_index = data.index.get_level_values('datetime_start')
    datetime_index = pd.DatetimeIndex(datetime_index)

    if complement:
        return data[(datetime_index.year != day.year) | (datetime_index.month != day.month) | (datetime_index.day != day.day)]
    else:
        return data[(datetime_index.year == day.year) & (datetime_index.month == day.month) & (datetime_index.day == day.day)]


In [ ]:
test_data = filter_for_day(data, test_day)
validation_data = filter_for_day(data, validation_day)
train_data = filter_for_day(data, test_day, complement=True)  # Exclude the test day...
train_data = filter_for_day(train_data, validation_day, complement=True)  # ... and the validation day.

In [ ]:
print(data.shape, train_data.shape, test_data.shape, validation_data.shape)

In [ ]:
features = ['site_hash', 'timestamp_start', 'precipitation mm/h', 'temperature C', 'windspeed m/s']
targets = ['trafficspeed km/h']#, 'trafficflow counts/h']

Note that `site_hash` is the `mmh3.hash64` of the `site` column (the last component actually):

In [ ]:
mmh3.hash64('rws01_monibas_0010vwa0056ra')[-1]

In [ ]:
# features.npartitions  # Only for dask dataframes

As you can see we (lazy) loaded the entire dataset. It has been distributed into the above number of partitions.

## Modeling

In [ ]:
import numpy as np
import theano
import theano.tensor as T
theano.config.exception_verbosity = 'high'
theano.config.floatX

In [ ]:
import lasagne

In [ ]:
theano.config.device

### Prepare batches

We model one sequence as whole-day measurement of a site. Batches are sets of such sequences of (possibly) various sizes. We expect a periodicity on the day level and try to fit a model to such behaviour.

To calculate the loss we could take the mean of the squared error over the whole sequence
but that would affect the lerning rate considerably. In the literature people tend to take
only the last sequence elements. For us *THIS IS NOT IDEAL SINCE OUR WE HAVE DAY-LONG MEASUREMENTS*
and the last element is with high probability around the end of the day. We would overwhelmingly train
to predict end of day traffic measurements.

We solve this by batching over *random* subsequences of the whole day sequences with a length controlled
by SUBSEQ_LENGTH.

We generate mini batches with the following function.

In [ ]:
from itertools import product

def batches(source_df, sites=None, days=None,
            max_batches=MAX_BATCHES, max_batch_length=MAX_BATCH_LENGTH, subseq_length=SUBSEQ_LENGTH):
    if sites is None:
        site_bag = set(source_df.index.get_level_values(0))
        
    if days is None:
        day_bag = set(source_df['day'].unique())
        
    sample_bag = product(sites, days)

    for i in range(max_batches):
        samples = list()
        for j in range(max_batch_length):
            try:
                samples.append(next(sample_bag))
            except StopIteration:
                break
        
        if len(samples) == 0:
#             print("No samples at batch %i" % i)
            raise StopIteration
        
        # Prepare batch
        batch_length = len(samples)
        batch = np.zeros([batch_length, subseq_length, len(features)], dtype='float64')
        mask = np.zeros([batch_length, subseq_length], dtype='float64')
        target = np.zeros([batch_length, subseq_length, len(targets)], dtype='float64')

        max_seq_length = 0
        for j in range(batch_length):
            site, period = samples[j]
            
            # query measurements
            data = source_df.query("site == '%s'" % site)
            data = data[data['day'] == period]
            
            data_f = data[features].values
            data_t = data[targets].values

            this_length = len(data)
            max_seq_length = max(max_seq_length, this_length)
            if this_length < subseq_length:
#                 print('Warning: batch to small: {}'.format(this_length))

                batch[j, :this_length, :] = data_f
                target[j, :this_length, :] = data_t
                mask[j, :this_length] = np.ones([this_length])
            else:
                # Select a random subsequence
                ind = np.random.random_integers(0, high=(this_length-subseq_length))
                data_f = data_f[ind:ind+subseq_length,:]
                data_t = data_t[ind:ind+subseq_length,:]

                batch[j, :, :] = data_f
                target[j, :, :] = data_t
                mask[j, :] = np.ones([subseq_length])
        
#         print('Max sequence length in this batch: {}'.format(max_seq_length))
        yield i, batch, target, mask


### Create the input and target variables

In [ ]:
input_var = T.tensor3('input', dtype=theano.config.floatX)

In [ ]:
target_tensor = T.tensor3('target', dtype=theano.config.floatX)

### Define the model

In [ ]:
l_in = lasagne.layers.InputLayer(shape=(None, None, len(features)), input_var=input_var, name='input_layer')

In [ ]:
l_mask = lasagne.layers.InputLayer(shape=(None,None), name='mask')

In [ ]:
help(lasagne.layers.LSTMLayer)

In [ ]:
num_lstm_units = 64
max_grad = 100.0
l_lstm_1 = lasagne.layers.LSTMLayer(l_in, num_units=num_lstm_units,
                                    hid_init=lasagne.init.GlorotUniform(), cell_init=lasagne.init.GlorotNormal(),
                                    gradient_steps=-1, grad_clipping=max_grad, unroll_scan=False,
                                    mask_input=l_mask, name='l_lstm_1')

In [ ]:
l_lstm_2 = lasagne.layers.LSTMLayer(l_lstm_1, num_units=num_lstm_units//4,
                                    hid_init=lasagne.init.GlorotUniform(), cell_init=lasagne.init.GlorotNormal(),
                                    gradient_steps=-1, grad_clipping=max_grad, unroll_scan=False,
                                    mask_input=l_mask, name='l_lstm_2')

In [ ]:
l_reshape_out = lasagne.layers.LSTMLayer(l_lstm_2, num_units=1,
                                         hid_init=lasagne.init.GlorotUniform(), cell_init=lasagne.init.GlorotNormal(),
                                         gradient_steps=-1, grad_clipping=max_grad, unroll_scan=False,
                                         mask_input=l_mask, name='l_lstm_out')

The following is no longer in use but here nontheless for future reference.

In [ ]:
# We want to combine the LSTM with a dense layer and need to reshape the input. We dot this with a `ReshapeLayer`
# help(lasagne.layers.ReshapeLayer)

In [ ]:
# Now, we can apply feed-forward layers as usual.
# l_dense_1 = lasagne.layers.DenseLayer(l_reshape_in, num_units=num_lstm_units, nonlinearity=lasagne.nonlinearities.tanh, name='l_dense_1')
# l_dense_2 = lasagne.layers.DenseLayer(l_dense_1, num_units=1, nonlinearity=lasagne.nonlinearities.tanh, name='l_dense_2')
# Now, the shape will be n_batch*n_timesteps, 1.  We can then reshape to
# n_batch, n_timesteps to get a single value for each timstep from each sequence
# l_reshape_out = lasagne.layers.ReshapeLayer(l_dense_2, (n_batch, n_time_steps, 1), name='output_layer')

### Function declarations

In [ ]:
print(lasagne.layers.get_output_shape(l_lstm_1))
print(lasagne.layers.get_output_shape(l_lstm_2))

print(lasagne.layers.get_output_shape(l_reshape_out))

In [ ]:
# lasagne.layers.get_output produces an expression for the output of the net
network_output = lasagne.layers.get_output(l_reshape_out)

In [ ]:
predicted_values = T.squeeze(network_output)

In [ ]:
# We do the same for the target_values
target_values = T.squeeze(target_tensor)

In [ ]:
# Our cost will be mean-squared error
# help(lasagne.objectives.squared_error)
loss = T.mean(lasagne.objectives.squared_error(predicted_values, target_values))
# loss = (predicted_values - target_values)**2

In [ ]:
# Retrieve all parameters from the network
all_params = lasagne.layers.get_all_params(l_reshape_out)
# all_params

In [ ]:
# Compute adam updates for training
updates = lasagne.updates.adam(loss, all_params, learning_rate=0.01)
# updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)

### Compile

Theano functions for training computing cost and inference

In [ ]:
train = theano.function([l_in.input_var, target_tensor, l_mask.input_var], loss, updates=updates)

In [ ]:
compute_cost = theano.function([l_in.input_var, target_tensor, l_mask.input_var], loss)

In [ ]:
ide = theano.function([target_tensor], outputs=[target_values])

The following is the feed-forward functions used for inference

In [ ]:
ff = theano.function([l_in.input_var, l_mask.input_var], outputs=[predicted_values])

### Check shapes

In [ ]:
validation_sites = set(validation_data.index.get_level_values(0))

In [ ]:
batch_num, batch, target, mask = next(iter(batches(validation_data, sites=validation_sites, days=(validation_day,))))

In [ ]:
print(batch.shape, target.shape, mask.shape)

In [ ]:
o = ff(batch, mask)[0]

In [ ]:
o[50,:20]

In [ ]:
o.shape

In [ ]:
target.shape

In [ ]:
t = ide(target)[0]

In [ ]:
t.shape

In [ ]:
compute_cost(batch, target, mask)

### Perform the training

This can take a long time beware!

In [ ]:
site_bag = list(set(train_data.index.get_level_values(0)))  # set gets rid of the duplicates
test_sites = list(set(test_data.index.get_level_values(0)))

In [ ]:
day_bag = list(train_data['day'].unique())

In [ ]:
print(len(site_bag), len(day_bag))

In [ ]:
from math import sqrt

In [ ]:
# We'll train the network with 10 epochs of a maximum of `max_batches` each
# Note the shuffle in each iteration of a batch
num_epochs = 10
for epoch in range(num_epochs):
    print('TRAIN Epoch {}'.format(epoch), end=' ')
    shuffle(site_bag)
    shuffle(day_bag)
    for batch_num, batch, target, mask in batches(train_data, sites=site_bag, days=day_bag):
        train(batch, target, mask)
        if batch_num % 10 == 0:
            if batch_num % 100 == 0:
                print(batch_num, end='')
            print(".", end='')
    
    print('')

    cost_test = 0.0
    print('TEST Epoch {}'.format(epoch), end=' ')
    for batch_num, batch, target, mask in batches(test_data, days=set((test_day,)), sites=test_sites):
        cost_test += compute_cost(batch, target, mask)
        if batch_num % 10 == 0:
            print(batch_num, end='')

    cost_test = cost_test/(batch_num + 1)
    print('')
    
    print("Epoch {} test cost (RMSE) = {}".format(epoch + 1, sqrt(cost_test)))

### Saving trained parameters

In [ ]:
def save(fpath):
    np.savez(fpath, *lasagne.layers.get_all_param_values(l_reshape_out))

def load(fpath):
    with np.load(fpath) as f:
        all_param = [f['arr_%d' % i] for i in range(len(f.files))]
    lasagne.layers.set_all_param_values(l_reshape_out, all_param)

In [ ]:
save('../src/predictor/models/lasagne_lstm_adam_0.01.npz')  # Give an identifying name please.

In [ ]:
load('../src/predictor/models/lasagne_lstm_adam_0.01.npz')

### Visualizing

In [ ]:
import theano.d3viz as d3v
from IPython.display import IFrame
d3v.d3viz(ff, '../tf_logs/lasagne_lstm.html')
IFrame('../tf_logs/lasagne_lstm.html', width=700, height=500)

### Validation

In [ ]:
validation_sites = set(validation_data.index.get_level_values(0))

In [ ]:
batch_num, batch, target, mask = next(iter(batches(validation_data, sites=validation_sites, days=(validation_day,))))

In [ ]:
compute_cost(batch, target, mask)

In [ ]:
features

In [ ]:
seq_sample = batch[60,:720,:]
seq_sample_target = target[60,:720,:]
seq_sample_mask = mask[60,:720]
print(seq_sample.shape)
print(seq_sample_target[:5,:])

In [ ]:
ff(seq_sample.reshape((1, *seq_sample.shape)), seq_sample_mask.reshape((1, *seq_sample_mask.shape)))